In [37]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [38]:
import ipynb.fs.full.TrainTest as TrainTest

p = 30 # want this to be high
w = 4  # want this to be low
cw = 9 # want this to be high
data = np.load('./data/' + 'p' + str(p) + '_w' + str(w) + '_cw' + str(cw) + '.npy')

x_train, y_train, x_test, y_test = TrainTest.get_train_test(data, 0.8)

Do RandomizedSearchCV

In [39]:
knn = KNeighborsClassifier()

param_options = {
    'n_neighbors' : range(1, 6),
    'weights' : ['uniform', 'distance'],
    'algorithm' : ['auto'],
    'leaf_size' : [10, 20, 30, 40, 50],
    'p' : [1, 2, 3, 4],
    'metric' : ['euclidean', 'manhattan', 'chebyshev', 'minkowski'],
#     'metric_params' : [],
#     'n_jobs' : []
}

knn_grid = GridSearchCV(estimator = knn, param_grid = param_options, cv = 10, verbose=1, n_jobs = -1)

knn_grid.fit(x_train, y_train)

Fitting 10 folds for each of 800 candidates, totalling 8000 fits


[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 1024 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 2524 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 4624 tasks      | elapsed:   38.5s
[Parallel(n_jobs=-1)]: Done 6037 tasks      | elapsed:   56.0s
[Parallel(n_jobs=-1)]: Done 7687 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 8000 out of 8000 | elapsed:  1.5min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'leaf_size': [10, 20, 30, 40, 50], 'algorithm': ['auto'], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski'], 'p': [1, 2, 3, 4], 'n_neighbors': range(1, 6)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [40]:
best = knn_grid.best_params_
best

{'algorithm': 'auto',
 'leaf_size': 10,
 'metric': 'euclidean',
 'n_neighbors': 1,
 'p': 1,
 'weights': 'uniform'}

In [41]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

knn = KNeighborsClassifier(**best)

avg_acc = 0
avg_prec = 0
avg_f1 = 0
num = 10
for i in range(num):
    knn.fit(x_train, y_train)
    
    y_pred = knn.predict(x_test)
    
    avg_acc += accuracy_score(y_test, y_pred)
    avg_prec += precision_score(y_test, y_pred)
    avg_f1 += f1_score(y_test, y_pred)

avg_acc /= num
avg_prec /= num
avg_f1 /= num

print('Average accuracy:  ' + str(avg_acc))
print('Average precision: ' + str(avg_prec))
print('Average F1 Score:  ' + str(avg_f1))

Average accuracy:  0.8202531645569622
Average precision: 0.7869565217391303
Average F1 Score:  0.8360277136258658
